In [71]:
import pandas as pd
TOI =  pd.read_csv('TOI_2022.04.12_07.30.35.csv', skiprows = 73)
TOI = TOI.drop(labels = ['tid', 'rastr', 'ra', 'decstr', 'dec', 'st_pmra', 'st_pmdec', 'toi_created', 'rowupdate'], axis =1 )
TOI = TOI.drop(['st_pmraerr1','st_pmraerr2', 'st_pmralim', 'st_pmdecerr1', 'st_pmdecerr2', 'st_pmdeclim'], axis = 1)
TOI = TOI.drop(['pl_tranmiderr1', 'pl_tranmiderr2', 'pl_tranmidlim','pl_orbpererr1', 'pl_orbpererr2', 'pl_orbperlim'], axis = 1)
TOI = TOI.drop(['pl_trandurherr1', 'pl_trandurherr2', 'pl_trandurhlim', 'pl_trandeperr1', 'pl_trandeperr2', 'pl_trandeplim', 'pl_radeerr1', 'pl_radeerr2', 'pl_radelim', 'pl_insolerr1', 'pl_insolerr2', 'pl_insollim', 'pl_eqterr1','pl_eqterr2', 'pl_eqtlim'], axis = 1)
TOI = TOI[TOI.columns.drop(list(TOI.filter(regex='err2')))]
TOI = TOI[TOI.columns.drop(list(TOI.filter(regex='err1')))]
TOI = TOI[TOI.columns.drop(list(TOI.filter(regex='lim')))]
Renamedict=  {'toi': 'Planet Identification Number', 'tfopwg_disp': 'Object Type', 'pl_tranmid':'Time to reach midpoint of transit', 'pl_orbper': 'Orbital Period', 'pl_trandurh': 'Total Length of Transit (H)', 'pl_trandep' : 'Depth of Light Curve', 'pl_rade' : 'Planet Radius (in Earth Radii)', 'pl_insol': 'Amount of Solar Sunlight recieved (relative to Earth)', 'pl_eqt': 'Average Planet Temperature (K)', 'st_tmag' : 'Star Brightness', 'st_dist': 'Distance to Star', 'st_teff': 'Star Temperature', 'st_logg':'Gravity at star surface','st_rad': 'Star Radius'}
TOI = TOI.rename(Renamedict, axis = 1)
TOI

,Planet Identification Number,Object Type,Time to reach midpoint of transit,Orbital Period,Total Length of Transit (H),Depth of Light Curve,Planet Radius (in Earth Radii),Amount of Solar Sunlight recieved (relative to Earth),Average Planet Temperature (K),Star Brightness,Distance to Star,Star Temperature,Gravity at star surface,Star Radius
0,1097.02,PC,2.459344e+06,13.902839,2.126443,647.270104,2.613685,77.484188,756.698759,8.7215,79.5561,5876.0,4.48000,0.983215
1,4405.02,PC,2.459037e+06,1.092938,1.628862,379.868608,1.360729,458.236934,1180.027890,12.3813,155.5010,4361.0,4.59663,0.687025
2,772.02,PC,2.459314e+06,NaN,4.456935,4273.827679,5.298360,5.322014,387.381397,10.8815,135.3710,5184.0,4.57383,0.815374
3,790.02,PC,2.458345e+06,41.017478,7.956864,234.153711,2.321129,62.461834,717.007061,8.9914,167.2180,6988.0,4.25649,1.460800
4,2000.02,PC,2.459283e+06,3.098196,2.244660,413.966764,2.529297,692.332942,1308.273362,10.3579,175.7130,5400.0,4.43000,1.180930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5483,120.01,CP,2.458332e+06,11.535076,8.555958,2389.906160,12.215808,NaN,NaN,7.1062,79.5702,5479.3,3.77000,2.487710
5484,128.01,PC,2.459385e+06,4.940468,1.966000,340.000000,2.218980,182.406000,1023.000000,7.9849,68.4727,6086.0,4.26269,1.130000
5485,130.01,CP,2.459369e+06,14.339206,4.586820,398.599933,2.314492,118.753936,841.941183,7.4371,57.5846,6295.7,4.43800,1.137420
5486,137.01,FA,2.459199e+06,4.817380,1.323000,1800.000000,1.711370,33.919000,672.000000,10.7020,88.8838,4651.0,4.56937,0.690000


In [80]:
import matplotlib.pyplot as plt
%matplotlib inline
TOI = TOI.drop(TOI.loc[TOI['Object Type']== 'APC'].index)
TOI = TOI.drop(TOI.loc[TOI['Object Type']== 'PC'].index)

TOI = TOI.replace(['CP', 'KP'], 1)
TOI = TOI.replace(['FA', 'FP'], 0)
TOI['Object Type'].value_counts() #We have a pretty even distibution of planets and non planets
plt.scatter

0.0    848
1.0    632
Name: Object Type, dtype: int64